In [1]:
import selenium
import json
import time
from urllib.request import urlopen
import re
import openpyxl
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
# 全局变量
with open('config.json', 'r', encoding='utf-8') as f:
    JsonFile = json.load(f)
base_url = JsonFile['base_url']
wait_time = JsonFile['wait_time']
delay = JsonFile['delay']
decade_year_xpaths = JsonFile['decade_year_xpaths']
year_value_xpaths = JsonFile['year_value_xpaths']
month_value_xpaths = JsonFile['month_value_xpaths']
txt_xpaths = JsonFile['txt_xpaths']
keyword1 = JsonFile['keyword1']
keyword2 = JsonFile['keyword2']
keyword3 = JsonFile['keyword3']
keyword4 = JsonFile['keyword4']
search = JsonFile['search']

In [3]:
# 打开网址并且下载txt文档
def open_and_download(url):
    
    wd = driver.get(url)
    # botton_decade_year = WebDriverWait(wd, 50, 0.5).until(EC.presence_of_element_located((By.XPATH,"""//*[@id="blockMarsBranding"]/a/img""")))
    try:
        WebDriverWait(wd, wait_time, 0.5).until(
            EC.presence_of_element_located((By.XPATH,decade_year_xpaths)))
    except:
        print("开始计时")
        time.sleep(delay)
        print("结束计时")
    bottons_decade_year = driver.find_elements(By.XPATH, decade_year_xpaths)
    for botton in bottons_decade_year:
        botton.click()
    print("已点开十年")

    bottons_year_value = driver.find_elements(By.XPATH, year_value_xpaths)
    for botton in bottons_year_value:
        botton.click()
        time.sleep(0.2)
    print("已点开每年")

    bottons_month_value = driver.find_elements(By.XPATH, month_value_xpaths)
    for botton in bottons_month_value:
        try:
            botton.click()
            time.sleep(5)
        except:
            time.sleep(5)
            botton.click()
            time.sleep(5)

    print("已点开每月")

def get_all_txt_url():
    hrefs = []
    bottons_txt = driver.find_elements(By.XPATH, txt_xpaths)
    for botton in bottons_txt:
        # print(botton.get_attribute('href'))
        hrefs.append(botton.get_attribute('href'))
    print(hrefs)
    print("已获取每份txt的URL")
    return hrefs

In [ ]:
if __name__ == '__main__':
    driver = webdriver.Chrome(r'.\chromedriver.exe')
    open_and_download(base_url)
    txt_url_list = get_all_txt_url()
    

C:\Users\Mokou\AppData\Local\Temp\ipykernel_13180\3383509022.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'.\chromedriver.exe')


开始计时
结束计时
已点开十年
已点开每年


In [15]:
# 处理好的文件
textPage=urlopen('https://mymarketnews.ams.usda.gov/filerepo/sites/default/files/3071/2017-11-06/469621/WA_GR10120171106.TXT')
content_bytes = textPage.read()
print(content_bytes)
content = str(content_bytes, encoding="utf8")
# 处理多余的字符，将他们全部转为空格
content = content.replace('\t', ' ').replace('\r', ' ').replace('\n', ' ')
# 将逗号句号转换为空
content = content.replace(',', '').replace('.', '')
# 合并多余空格
content = re.sub(' +', ' ', content)

print(content)

b"WA_GR101\r\nWashington, DC    Mon   Nov 6, 2017   USDA Market News\r\n\r\n*CORRECTIONS AT BOTTOM OF REPORT ARE INCLUDED IN CURRENT MARKET YEAR TO DATE*\r\n\r\nCOUNTRY OF DESTINATION IS REPORTED AS KNOWN AT THE TIME OF EXPORTATION.\r\nINFORMATION CONTAINED IN THIS REPORT REFLECTS EXPORTED GRAIN INSPECTED AND \r\nWEIGHED THROUGH GIPSA'S AUTHORITY UNDER THE U.S. GRAIN STANDARDS ACT.  \r\nNO ADDITIONAL ANALYSIS, COMPILATIONS OR DATA IS AVAILABLE.\r\n\r\n               GRAINS INSPECTED AND/OR WEIGHED FOR EXPORT\r\n                  REPORTED IN WEEK ENDING NOV 02, 2017\r\n                            -- METRIC TONS --\r\n-------------------------------------------------------------------------\r\n                                                   CURRENT     PREVIOUS  \r\n             ----------- WEEK ENDING ----------  MARKET YEAR  MARKET YEAR\r\n  GRAIN      11/02/2017  10/26/2017  11/03/2016    TO DATE     TO DATE   \r\n\r\nBARLEY            122         636         612       16,006      

In [22]:
temp_content = content
print("分割前\n", temp_content)
if keyword1 != "":
    content_spilt = re.split(keyword1, temp_content)
    temp_content = content_spilt[1]
print("第一次分割\n", temp_content)
if keyword2 != "":
    content_spilt = re.split(keyword2, temp_content)
    temp_content = content_spilt[0]
print("第二次分割\n", temp_content)

分割前
 WA_GR101 Washington DC Mon Nov 6 2017 USDA Market News *CORRECTIONS AT BOTTOM OF REPORT ARE INCLUDED IN CURRENT MARKET YEAR TO DATE* COUNTRY OF DESTINATION IS REPORTED AS KNOWN AT THE TIME OF EXPORTATION INFORMATION CONTAINED IN THIS REPORT REFLECTS EXPORTED GRAIN INSPECTED AND WEIGHED THROUGH GIPSA'S AUTHORITY UNDER THE US GRAIN STANDARDS ACT NO ADDITIONAL ANALYSIS COMPILATIONS OR DATA IS AVAILABLE GRAINS INSPECTED AND/OR WEIGHED FOR EXPORT REPORTED IN WEEK ENDING NOV 02 2017 -- METRIC TONS -- ------------------------------------------------------------------------- CURRENT PREVIOUS ----------- WEEK ENDING ---------- MARKET YEAR MARKET YEAR GRAIN 11/02/2017 10/26/2017 11/03/2016 TO DATE TO DATE BARLEY 122 636 612 16006 25605 CORN 444648 547417 929281 5540404 10177385 FLAXSEED 122 220 0 4111 9508 MIXED 0 0 0 0 0 OATS 0 0 0 3893 6299 RYE 0 0 0 0 0 SORGHUM 53476 1485 82363 499562 692698 SOYBEANS 2490600 2528098 2677810 14856207 16318565 SUNFLOWER 0 0 0 0 0 WHEAT 284293 318696 512403

In [17]:
temp_content = temp_content.replace('CHINA T', 'TAIWAN')
temp_content = temp_content.replace('CHINA MAIN', 'CHINA')
print(temp_content)

 REPORTED IN WEEK ENDING NOV 02 2017 -- METRIC TONS -- --------------------------------------------------------------------------------------- CORN CORN FLAX- REGION/COUNTRY WHEAT YELLOW WHITE BARLEY SORGHUM SOYBEANS SEED TOTALS LAKES BELGIUM 0 0 0 0 0 21317 0 21317 SUBTOTAL 0 0 0 0 0 21317 0 21317 ATLANTIC CHINA 930 0 0 0 0 50909 0 51839 TAIWAN 0 0 0 0 0 12222 0 12222 INDONESIA 0 0 0 0 0 12070 0 12070 JAMAICA 0 4646 0 0 0 0 0 4646 KOREA REP 392 0 0 0 0 0 0 392 MALAYSIA 0 0 0 0 0 3551 0 3551 THAILAND 0 0 0 0 0 3012 0 3012 VIETNAM 0 0 0 0 0 147 0 147 SUBTOTAL 1322 4646 0 0 0 81911 0 87879 GULF CHINA 0 0 0 0 50799 1032544 0 1083343 COLOMBIA 0 129940 8000 0 0 11760 0 149700 EGYPT 0 0 0 0 0 66399 0 66399 HAITI 0 6900 0 0 0 0 0 6900 HONDURAS 0 38528 0 0 0 0 0 38528 JAMAICA 0 6950 0 0 0 0 0 6950 JAPAN 0 69501 0 0 0 29869 0 99370 KOREA REP 0 0 0 0 0 57212 0 57212 MEXICO 12851 55984 0 0 0 65903 0 134738 NETHERLANDS 0 0 0 0 0 84617 0 84617 NIGERIA 33000 0 0 0 0 0 0 33000 PERU 0 35355 0 0 0 0 0 

In [18]:
temp_keyword = temp_content
print("分割前\n", temp_keyword)
if keyword3 != "":
    content_spilt = re.split(keyword3, temp_keyword)
    temp_keyword = content_spilt[1]
print("第一次分割\n", temp_keyword)
if keyword4 != "":
    content_spilt = re.split(keyword4, temp_keyword)
    temp_keyword = content_spilt[0]
print("第二次分割\n", temp_keyword)


分割前
  REPORTED IN WEEK ENDING NOV 02 2017 -- METRIC TONS -- --------------------------------------------------------------------------------------- CORN CORN FLAX- REGION/COUNTRY WHEAT YELLOW WHITE BARLEY SORGHUM SOYBEANS SEED TOTALS LAKES BELGIUM 0 0 0 0 0 21317 0 21317 SUBTOTAL 0 0 0 0 0 21317 0 21317 ATLANTIC CHINA 930 0 0 0 0 50909 0 51839 TAIWAN 0 0 0 0 0 12222 0 12222 INDONESIA 0 0 0 0 0 12070 0 12070 JAMAICA 0 4646 0 0 0 0 0 4646 KOREA REP 392 0 0 0 0 0 0 392 MALAYSIA 0 0 0 0 0 3551 0 3551 THAILAND 0 0 0 0 0 3012 0 3012 VIETNAM 0 0 0 0 0 147 0 147 SUBTOTAL 1322 4646 0 0 0 81911 0 87879 GULF CHINA 0 0 0 0 50799 1032544 0 1083343 COLOMBIA 0 129940 8000 0 0 11760 0 149700 EGYPT 0 0 0 0 0 66399 0 66399 HAITI 0 6900 0 0 0 0 0 6900 HONDURAS 0 38528 0 0 0 0 0 38528 JAMAICA 0 6950 0 0 0 0 0 6950 JAPAN 0 69501 0 0 0 29869 0 99370 KOREA REP 0 0 0 0 0 57212 0 57212 MEXICO 12851 55984 0 0 0 65903 0 134738 NETHERLANDS 0 0 0 0 0 84617 0 84617 NIGERIA 33000 0 0 0 0 0 0 33000 PERU 0 35355 0 0 0

In [19]:
# 返回要查找数据的列数
list_keyword = temp_keyword.split()
number_location = 0
print(list_keyword)
number_location=list_keyword.index(search)
if search == "TOTALS":
    number_location = len(list_keyword)
print(number_location)

['WHEAT', 'YELLOW', 'WHITE', 'BARLEY', 'SORGHUM', 'SOYBEANS', 'SEED']
2


In [23]:
temp_content = temp_content.split('METRIC TONS')[1]
print(temp_content)
num_list = re.findall('\d+', str(temp_content))
print(num_list)


 -- --------------------------------------------------------------------------------------- CORN CORN FLAX- REGION/COUNTRY WHEAT YELLOW WHITE BARLEY SORGHUM SOYBEANS SEED TOTALS LAKES BELGIUM 0 0 0 0 0 21317 0 21317 SUBTOTAL 0 0 0 0 0 21317 0 21317 ATLANTIC CHINA MAIN 930 0 0 0 0 50909 0 51839 CHINA T 0 0 0 0 0 12222 0 12222 INDONESIA 0 0 0 0 0 12070 0 12070 JAMAICA 0 4646 0 0 0 0 0 4646 KOREA REP 392 0 0 0 0 0 0 392 MALAYSIA 0 0 0 0 0 3551 0 3551 THAILAND 0 0 0 0 0 3012 0 3012 VIETNAM 0 0 0 0 0 147 0 147 SUBTOTAL 1322 4646 0 0 0 81911 0 87879 GULF CHINA MAIN 0 0 0 0 50799 1032544 0 1083343 COLOMBIA 0 129940 8000 0 0 11760 0 149700 EGYPT 0 0 0 0 0 66399 0 66399 HAITI 0 6900 0 0 0 0 0 6900 HONDURAS 0 38528 0 0 0 0 0 38528 JAMAICA 0 6950 0 0 0 0 0 6950 JAPAN 0 69501 0 0 0 29869 0 99370 KOREA REP 0 0 0 0 0 57212 0 57212 MEXICO 12851 55984 0 0 0 65903 0 134738 NETHERLANDS 0 0 0 0 0 84617 0 84617 NIGERIA 33000 0 0 0 0 0 0 33000 PERU 0 35355 0 0 0 0 0 35355 SAUDI ARABIA 0 0 0 0 0 71498 0 714

In [28]:
num = len(list_keyword)
while True:
    print(num_list[num])
    num = num + len(list_keyword) + 1
    if num > len(num_list):
        break

21317
21317
51839
12222
12070
4646
392
3551
3012
147
87879
1083343
149700
66399
6900
38528
6950
99370
57212
134738
84617
33000
35355
71498
3345
1870955
740670
29056
49590
58941
33000
35452
563
23934
971206
1224
30070
39333
490
245
28528
1444
2937
5703
169699
342
5290
6014
291319
3242676
30585
